In [0]:
from delta.tables import DeltaTable

In [0]:
customer_target_table = "workspace.zoom_car.dim_customers"
booking_target_table = "workspace.zoom_car.fact_bookings"

staging_customer_path = "workspace.zoom_car.staging_customers_delta"
staging_booking_path = "workspace.zoom_car.staging_bookings_delta"


In [0]:
from delta.tables import DeltaTable

# MERGE for Customers
if spark.catalog.tableExists(customer_target_table):
    target_customers = DeltaTable.forName(spark, customer_target_table)
    (
        target_customers.alias("t")
        .merge(
            spark.table(staging_customer_path).alias("s"),
            "t.customer_id = s.customer_id"
        )
        .whenMatchedUpdateAll()
        .whenNotMatchedInsertAll()
        .execute()
    )
else:
    spark.table(staging_customer_path) \
        .write.format("delta") \
        .mode("overwrite") \
        .saveAsTable(customer_target_table)



In [0]:
# Load staging data
staging_df = spark.table(staging_booking_path)

# Check if target table exists
if spark.catalog.tableExists(booking_target_table):
    target_bookings = DeltaTable.forName(spark, booking_target_table)
    
    (
        target_bookings.alias("t")
        .merge(
            staging_df.alias("s"),
            "t.booking_id = s.booking_id"
        )
        .whenMatchedUpdateAll(condition="s.status != 'cancelled'")  # Update active bookings
        .whenMatchedDelete(condition="s.status = 'cancelled'")      # Delete cancelled bookings
        .whenNotMatchedInsertAll(condition="s.status != 'cancelled'")  # Insert only active bookings
        .execute()
    )

else:
    # First load: Insert only non-cancelled bookings
    staging_df.filter("status != 'cancelled'") \
        .write.format("delta") \
        .mode("overwrite") \
        .saveAsTable(booking_target_table)


In [0]:
display(spark.sql(f"select * from {customer_target_table}"))
display(spark.sql(f"select * from {booking_target_table}"))

customer_id,email,name,phone_number,signup_date,status,tenure_days,arrival_date,load_timestamp
C001,joshuathomas@yahoo.com,Emma Hawkins,3138135510,2025-05-28,inactive,50,2025-07-17,2025-07-17T17:48:32.995Z
C002,lrosario@gmail.com,Brenda Galloway,3304466710,2025-03-28,active,111,2025-07-17,2025-07-17T17:48:32.995Z
C003,yanderson@gmail.com,Michele Branch,1895122579,2025-02-17,active,150,2025-07-17,2025-07-17T17:48:32.995Z
C004,rhale@gmail.com,Jessica Gray,7041080388,2024-10-28,active,262,2025-07-17,2025-07-17T17:48:32.995Z
C005,orobinson@gmail.com,Carolyn Martinez,1403054963,2025-06-25,active,22,2025-07-17,2025-07-17T17:48:32.995Z
C006,twilliams@simmons.com,Kathy Strickland,8385627187,2024-11-14,active,245,2025-07-17,2025-07-17T17:48:32.995Z
C007,aramirez@vazquez.com,Tabitha Smith,4917908181,2025-02-08,active,159,2025-07-17,2025-07-17T17:48:32.995Z
C009,hcabrera@hotmail.com,Rebecca Fleming,5078290394,2024-03-13,active,491,2025-07-17,2025-07-17T17:48:32.995Z
C010,tgoodwin@cohen.biz,Amanda Bradley,7593852622,2024-06-04,inactive,408,2025-07-17,2025-07-17T17:48:32.995Z


booking_date,booking_id,car_id,customer_id,end_time,start_time,status,total_amount,start_date,start_clock,end_date,end_clock,arrival_date,load_timestamp
2025-06-22T00:00:00.000Z,B042,CAR379,C002,2025-06-22T07:02:29.000Z,2025-06-22T01:02:29.000Z,completed,117.08,2025-06-22,01:02:29,2025-06-22,07:02:29,2025-07-17,2025-07-17T17:51:32.337Z
2025-07-10T00:00:00.000Z,B034,CAR681,C010,2025-07-10T02:11:59.000Z,2025-07-10T01:11:59.000Z,completed,388.86,2025-07-10,01:11:59,2025-07-10,02:11:59,2025-07-17,2025-07-17T17:51:32.337Z
2025-06-20T00:00:00.000Z,B021,CAR305,C002,2025-06-20T23:06:53.000Z,2025-06-20T18:06:53.000Z,pending,397.49,2025-06-20,18:06:53,2025-06-20,23:06:53,2025-07-17,2025-07-17T17:51:32.337Z
2025-06-23T00:00:00.000Z,B052,CAR848,C004,2025-06-23T11:23:00.000Z,2025-06-23T01:23:00.000Z,pending,358.4,2025-06-23,01:23:00,2025-06-23,11:23:00,2025-07-17,2025-07-17T17:51:32.337Z
2025-07-13T00:00:00.000Z,B023,CAR526,C004,2025-07-14T00:47:22.000Z,2025-07-13T12:47:22.000Z,pending,368.01,2025-07-13,12:47:22,2025-07-14,00:47:22,2025-07-17,2025-07-17T17:51:32.337Z
2025-07-13T00:00:00.000Z,B058,CAR314,C003,2025-07-13T09:56:14.000Z,2025-07-13T08:56:14.000Z,completed,187.69,2025-07-13,08:56:14,2025-07-13,09:56:14,2025-07-17,2025-07-17T17:51:32.337Z
2025-06-23T00:00:00.000Z,B081,CAR981,C007,2025-06-23T18:19:08.000Z,2025-06-23T07:19:08.000Z,pending,82.42,2025-06-23,07:19:08,2025-06-23,18:19:08,2025-07-17,2025-07-17T17:51:32.337Z
2025-07-15T00:00:00.000Z,B082,CAR948,C001,2025-07-15T19:17:58.000Z,2025-07-15T10:17:58.000Z,completed,177.35,2025-07-15,10:17:58,2025-07-15,19:17:58,2025-07-17,2025-07-17T17:51:32.337Z
2025-06-24T00:00:00.000Z,B084,CAR888,C003,2025-06-24T21:23:39.000Z,2025-06-24T17:23:39.000Z,pending,308.3,2025-06-24,17:23:39,2025-06-24,21:23:39,2025-07-17,2025-07-17T17:51:32.337Z
2025-07-08T00:00:00.000Z,B020,CAR134,C004,2025-07-09T03:41:27.000Z,2025-07-08T20:41:27.000Z,completed,222.79,2025-07-08,20:41:27,2025-07-09,03:41:27,2025-07-17,2025-07-17T17:51:32.337Z
